In [3]:
import boto3
dynamodb = boto3.resource('dynamodb')
question_table = dynamodb.Table('question')
user_table = 'user-profile'

In [70]:
from boto3.dynamodb.conditions import Key, Attr, And, Not, Or

In [5]:
user_id = 'sy2938@columbia.edu'

In [88]:
query = question_table.query(
        Limit = 5,
         IndexName="user_id",
        KeyConditionExpression=Key('user_id').eq(user_id)
        )

In [171]:
answer_query = question_table.query(
        Limit = 10,
        ScanIndexForward= False,
         IndexName="tutor_id",
        KeyConditionExpression=Key('tutor_id').eq(user_id)
        )

In [172]:
import pandas as pd

In [179]:
answer_query['Items'][0]['end_time']

'2021-04-20T20:00:00'

In [87]:
ask_query.get('LastEvaluatedKey')

{'question_id': '6dc98d16-9908-11eb-8788-acde48001122',
 'user_id': 'sy2938@columbia.edu',
 'created_at': Decimal('1617904800')}

In [197]:
d = answer_query['Items'][0]

In [198]:
start = datetime.datetime.fromisoformat(d['start_time'])
end = datetime.datetime.fromisoformat(d['end_time'])

'19:00-20:00 04/20/21'

In [203]:
f'{}-{} {}'

SyntaxError: invalid syntax (<ipython-input-203-f1869fe9240b>, line 1)

'04/20/21'

'20:00'

'19:00'

In [123]:
import json
import boto3
from decimal import Decimal
from boto3.dynamodb.conditions import Key, Attr, And, Not, Or
dynamodb = boto3.resource('dynamodb')
user_table = 'user-profile'
question_table = dynamodb.Table('question')
def lambda_handler(event, context):
    is_ask = event['headers']['is_ask']
    last_key = None
    access_token = event['headers']['access_token']
    try:
        resp = cognito.get_user(
            AccessToken=access_token,
        )
    except:
        return {
            'statusCode': 500,
            'body': json.dumps('Error in your login'),
            "headers": {
            'Content-Type':  'application/json',
            "Access-Control-Allow-Origin": "*",
            "Access-Control-Allow-Headers": "*"
            }
        }
    user = {i['Name']:i['Value'] for i in resp['UserAttributes']}
    user_id = user['email']


    if  event['headers']['last_key'] != 'null':
        last_key = json.loads(event['headers']['last_key'])
        last_key['created_at'] = Decimal(last_key['created_at'])
    
    
    if is_ask == 'true':
        if last_key:
            query = question_table.query(
                    Limit = 10,
                    ExclusiveStartKey=last_key,
                     IndexName="user_id",
                    KeyConditionExpression=Key('user_id').eq(user_id)
                )
        else:
            query = question_table.query(
                    Limit = 10,
                     IndexName="user_id",
                    KeyConditionExpression=Key('user_id').eq(user_id)
                )
    else:
        if last_key:
            query = question_table.query(
                    Limit = 10,
                    ExclusiveStartKey=last_key,
                     IndexName="tutor_id",
                    KeyConditionExpression=Key('tutor_id').eq(user_id)
                )
        else:
            query = question_table.query(
                    Limit = 10,
                     IndexName="tutor_id",
                    KeyConditionExpression=Key('tutor_id').eq(user_id)
                )
    data = query.get('Items')
    if not data:
        return {
            'statusCode': 200,
            'body': json.dumps({
                        'data': []
                    }),
            "headers": {
            'Content-Type':  'application/json',
            "Access-Control-Allow-Origin": "*",
            "Access-Control-Allow-Headers": "*"
            }
        }
    
    if len(data)!=0:
        user_ids = [i['user_id']for i in query['Items']] + [i['tutor_id']for i in query['Items'] if i['tutor_id'] !='Null']
        result = {i:{} for i in set(user_ids)}
        response = dynamodb.batch_get_item(
            RequestItems={
                user_table:{
                    "Keys": [{'user_id': user_id} for user_id in set(user_ids)],
                    'AttributesToGet': ['picture', 'first_name', 'last_name']
                }
            }
        )['Responses'][user_table]
        for email, res in zip(result, response):
            result[email] = res
    for d in data:
        d['created_at'] = int(d['created_at'])
        d['user_picture'] = result[d['user_id']]['picture']
        d['user'] = result[d['user_id']]['first_name'] + ' ' + result[d['user_id']]['last_name']
        d['status'] = d['question_status']
        if d['tutor_id'] != 'Null':
            d['tutor_picture'] = result[d['tutor_id']]['picture']
            d['tutor'] = result[d['tutor_id']]['first_name'] + ' ' + result[d['tutor_id']]['last_name']
        else:
            d['tutor_id'] = None
            d['tutor_picture'] = None
            d['tutor'] = None
        d.pop('question_status', None)
        if 'tags' not in d:
            d['tags'] = []

    if 'LastEvaluatedKey' in query:
        last_key = query['LastEvaluatedKey']
        last_key['created_at'] = int(last_key['created_at'])
        body = {
            'data': data,
            'LastEvaluatedKey': last_key
        }
    else:
        body = {
            'data': data
        }
    return {
        'statusCode': 200,
        'body': json.dumps(body),
        "headers": {
        'Content-Type':  'application/json',
        "Access-Control-Allow-Origin": "*",
        "Access-Control-Allow-Headers": "*"
        }
    }

In [124]:
event = {'resource': '/user/history', 'path': '/user/history', 'httpMethod': 'GET', 'headers': {'accept': '*/*', 'accept-encoding': 'gzip, deflate, br', 'accept-language': 'zh,en-US;q=0.9,en;q=0.8', 'access-control-allow-credentials': 'true', 'access-control-allow-headers': '*', 'access-control-allow-methods': 'GET', 'access-control-allow-origin': '*', 'access_token': 'eyJraWQiOiJFa3lrc1p4c0IzSDVtaDV0VkgzZ1M2VGlVNnVRXC9KbTNkZUd2U3F1WlNvcz0iLCJhbGciOiJSUzI1NiJ9.eyJzdWIiOiJkYWU5M2ZmZC05OWQ5LTRkYzMtOGI3NS1kMzAxMWVhZTc3MmMiLCJ0b2tlbl91c2UiOiJhY2Nlc3MiLCJzY29wZSI6ImF3cy5jb2duaXRvLnNpZ25pbi51c2VyLmFkbWluIHBob25lIG9wZW5pZCBwcm9maWxlIGVtYWlsIiwiYXV0aF90aW1lIjoxNjE3OTU2NzU0LCJpc3MiOiJodHRwczpcL1wvY29nbml0by1pZHAudXMtZWFzdC0xLmFtYXpvbmF3cy5jb21cL3VzLWVhc3QtMV9VSzZzTERENVUiLCJleHAiOjE2MTc5NjAzNTQsImlhdCI6MTYxNzk1Njc1NCwidmVyc2lvbiI6MiwianRpIjoiMGQwOGFhYTEtNWQ4Yy00MDdhLWI3MTAtMmVmYTE4ZGUyZjdhIiwiY2xpZW50X2lkIjoiMWQxbWIya3RmYXA5OGhnaWYxaWlnamI5ZmsiLCJ1c2VybmFtZSI6Ikdvb2dsZV8xMDg0MDE3NzIzNDMxNzEyODk0MzAifQ.t4ShihsVjgOEWtkXguCng0kbClA0di1HwOWijAzYhnZbyYBQl0WdGd1zUolkPmTf9yfkEcy5rexghn7Bw7pQTn12BD5848mutKQ0d3NspZ5fVZneVfcqXyxBqqNfFpQp5zLBSmyHNrJ6TNey3gSTqHc5e9_8pCUsCpWQ66drMhlstOCH9rYIjV-tjP6TCD3OBadCvzyvKGOfEP9gU388XrtncJFk-RlzoBw6ItJgbptLLJ6ZXvmMVtDSF_Khd3e6SbZYhFmIOVV-drs3WGPqhJ_M75NQzIz3MKzJQfucnSl5YCvr2kHepr1zi5NBQFAEm--E2fv4F2JkGW0b9GmUmw', 'content-type': 'application/json', 'Host': 'gr73qrcwnl.execute-api.us-east-1.amazonaws.com', 'is_ask': 'true', 'origin': 'http://localhost:3000', 'referer': 'http://localhost:3000/', 'sec-fetch-dest': 'empty', 'sec-fetch-mode': 'cors', 'sec-fetch-site': 'cross-site', 'sec-gpc': '1', 'token': 'eyJraWQiOiJ2cGJhcTR1R082R0RTYjRwUncwVEZnOXJDWjIyMittdkZHejFtRkRReVBJPSIsImFsZyI6IlJTMjU2In0.eyJhdF9oYXNoIjoiMFVkMExRVVAtcDAzY1NwWGZ4Y0dDZyIsInN1YiI6ImRhZTkzZmZkLTk5ZDktNGRjMy04Yjc1LWQzMDExZWFlNzcyYyIsImVtYWlsX3ZlcmlmaWVkIjpmYWxzZSwiaXNzIjoiaHR0cHM6XC9cL2NvZ25pdG8taWRwLnVzLWVhc3QtMS5hbWF6b25hd3MuY29tXC91cy1lYXN0LTFfVUs2c0xERDVVIiwiY29nbml0bzp1c2VybmFtZSI6Ikdvb2dsZV8xMDg0MDE3NzIzNDMxNzEyODk0MzAiLCJnaXZlbl9uYW1lIjoiSmFja2llIiwicGljdHVyZSI6Imh0dHBzOlwvXC9saDUuZ29vZ2xldXNlcmNvbnRlbnQuY29tXC8tVkZfY0t5Zm13OUVcL0FBQUFBQUFBQUFJXC9BQUFBQUFBQUFEOFwvQU1adXVjbHVNTzZWQS1HZG1LTU1sQWNhS24tR2VJZkZCd1wvczk2LWNcL3Bob3RvLmpwZyIsImF1ZCI6IjFkMW1iMmt0ZmFwOThoZ2lmMWlpZ2piOWZrIiwiaWRlbnRpdGllcyI6W3sidXNlcklkIjoiMTA4NDAxNzcyMzQzMTcxMjg5NDMwIiwicHJvdmlkZXJOYW1lIjoiR29vZ2xlIiwicHJvdmlkZXJUeXBlIjoiR29vZ2xlIiwiaXNzdWVyIjpudWxsLCJwcmltYXJ5IjoidHJ1ZSIsImRhdGVDcmVhdGVkIjoiMTYxNzA4NDE3NzYxNSJ9XSwidG9rZW5fdXNlIjoiaWQiLCJhdXRoX3RpbWUiOjE2MTc5NTY3NTQsImV4cCI6MTYxNzk2MDM1NCwiaWF0IjoxNjE3OTU2NzU0LCJmYW1pbHlfbmFtZSI6Ill1YW4iLCJlbWFpbCI6InN5MjkzOEBjb2x1bWJpYS5lZHUifQ.DR_thleo2E7OvIAG1bXHxEwjzqmPuIvVhfR73exLN20U9PljJJ9kvjfiZY82-8AMTKnJHpAO-d0KkTPJWdEv9ZABJVHiLTsrUnUohwenQbD59FmjJ5VL1fWBfLGYb9CvQsQihI5JtU0MwyGTOjE8vy0YgRBfob-no-HZK0zg9Wm5qgpJHdiWJzbKd19kjCICCVUzKC64888KX650Fy7TINh1o8nYmovFevD27lfRRGDw1oWschT4F6koslisbx2eGAAYZOwl3DNh6iAhbbdmJ_cUbP9sCCPWnMYKRun-bnb5sGUxvlL1Z6VfvyN39qS0jcffm8Pup4hpeCpKrDhfxw', 'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 11_2_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36', 'X-Amzn-Trace-Id': 'Root=1-60700f96-0ed0394d52ef733502b77e98', 'X-Forwarded-For': '100.33.115.103', 'X-Forwarded-Port': '443', 'X-Forwarded-Proto': 'https'}, 'multiValueHeaders': {'accept': ['*/*'], 'accept-encoding': ['gzip, deflate, br'], 'accept-language': ['zh,en-US;q=0.9,en;q=0.8'], 'access-control-allow-credentials': ['true'], 'access-control-allow-headers': ['*'], 'access-control-allow-methods': ['GET'], 'access-control-allow-origin': ['*'], 'access_token': ['eyJraWQiOiJFa3lrc1p4c0IzSDVtaDV0VkgzZ1M2VGlVNnVRXC9KbTNkZUd2U3F1WlNvcz0iLCJhbGciOiJSUzI1NiJ9.eyJzdWIiOiJkYWU5M2ZmZC05OWQ5LTRkYzMtOGI3NS1kMzAxMWVhZTc3MmMiLCJ0b2tlbl91c2UiOiJhY2Nlc3MiLCJzY29wZSI6ImF3cy5jb2duaXRvLnNpZ25pbi51c2VyLmFkbWluIHBob25lIG9wZW5pZCBwcm9maWxlIGVtYWlsIiwiYXV0aF90aW1lIjoxNjE3OTU2NzU0LCJpc3MiOiJodHRwczpcL1wvY29nbml0by1pZHAudXMtZWFzdC0xLmFtYXpvbmF3cy5jb21cL3VzLWVhc3QtMV9VSzZzTERENVUiLCJleHAiOjE2MTc5NjAzNTQsImlhdCI6MTYxNzk1Njc1NCwidmVyc2lvbiI6MiwianRpIjoiMGQwOGFhYTEtNWQ4Yy00MDdhLWI3MTAtMmVmYTE4ZGUyZjdhIiwiY2xpZW50X2lkIjoiMWQxbWIya3RmYXA5OGhnaWYxaWlnamI5ZmsiLCJ1c2VybmFtZSI6Ikdvb2dsZV8xMDg0MDE3NzIzNDMxNzEyODk0MzAifQ.t4ShihsVjgOEWtkXguCng0kbClA0di1HwOWijAzYhnZbyYBQl0WdGd1zUolkPmTf9yfkEcy5rexghn7Bw7pQTn12BD5848mutKQ0d3NspZ5fVZneVfcqXyxBqqNfFpQp5zLBSmyHNrJ6TNey3gSTqHc5e9_8pCUsCpWQ66drMhlstOCH9rYIjV-tjP6TCD3OBadCvzyvKGOfEP9gU388XrtncJFk-RlzoBw6ItJgbptLLJ6ZXvmMVtDSF_Khd3e6SbZYhFmIOVV-drs3WGPqhJ_M75NQzIz3MKzJQfucnSl5YCvr2kHepr1zi5NBQFAEm--E2fv4F2JkGW0b9GmUmw'], 'content-type': ['application/json'], 'Host': ['gr73qrcwnl.execute-api.us-east-1.amazonaws.com'], 'is_ask': ['true'], 'origin': ['http://localhost:3000'], 'referer': ['http://localhost:3000/'], 'sec-fetch-dest': ['empty'], 'sec-fetch-mode': ['cors'], 'sec-fetch-site': ['cross-site'], 'sec-gpc': ['1'], 'token': ['eyJraWQiOiJ2cGJhcTR1R082R0RTYjRwUncwVEZnOXJDWjIyMittdkZHejFtRkRReVBJPSIsImFsZyI6IlJTMjU2In0.eyJhdF9oYXNoIjoiMFVkMExRVVAtcDAzY1NwWGZ4Y0dDZyIsInN1YiI6ImRhZTkzZmZkLTk5ZDktNGRjMy04Yjc1LWQzMDExZWFlNzcyYyIsImVtYWlsX3ZlcmlmaWVkIjpmYWxzZSwiaXNzIjoiaHR0cHM6XC9cL2NvZ25pdG8taWRwLnVzLWVhc3QtMS5hbWF6b25hd3MuY29tXC91cy1lYXN0LTFfVUs2c0xERDVVIiwiY29nbml0bzp1c2VybmFtZSI6Ikdvb2dsZV8xMDg0MDE3NzIzNDMxNzEyODk0MzAiLCJnaXZlbl9uYW1lIjoiSmFja2llIiwicGljdHVyZSI6Imh0dHBzOlwvXC9saDUuZ29vZ2xldXNlcmNvbnRlbnQuY29tXC8tVkZfY0t5Zm13OUVcL0FBQUFBQUFBQUFJXC9BQUFBQUFBQUFEOFwvQU1adXVjbHVNTzZWQS1HZG1LTU1sQWNhS24tR2VJZkZCd1wvczk2LWNcL3Bob3RvLmpwZyIsImF1ZCI6IjFkMW1iMmt0ZmFwOThoZ2lmMWlpZ2piOWZrIiwiaWRlbnRpdGllcyI6W3sidXNlcklkIjoiMTA4NDAxNzcyMzQzMTcxMjg5NDMwIiwicHJvdmlkZXJOYW1lIjoiR29vZ2xlIiwicHJvdmlkZXJUeXBlIjoiR29vZ2xlIiwiaXNzdWVyIjpudWxsLCJwcmltYXJ5IjoidHJ1ZSIsImRhdGVDcmVhdGVkIjoiMTYxNzA4NDE3NzYxNSJ9XSwidG9rZW5fdXNlIjoiaWQiLCJhdXRoX3RpbWUiOjE2MTc5NTY3NTQsImV4cCI6MTYxNzk2MDM1NCwiaWF0IjoxNjE3OTU2NzU0LCJmYW1pbHlfbmFtZSI6Ill1YW4iLCJlbWFpbCI6InN5MjkzOEBjb2x1bWJpYS5lZHUifQ.DR_thleo2E7OvIAG1bXHxEwjzqmPuIvVhfR73exLN20U9PljJJ9kvjfiZY82-8AMTKnJHpAO-d0KkTPJWdEv9ZABJVHiLTsrUnUohwenQbD59FmjJ5VL1fWBfLGYb9CvQsQihI5JtU0MwyGTOjE8vy0YgRBfob-no-HZK0zg9Wm5qgpJHdiWJzbKd19kjCICCVUzKC64888KX650Fy7TINh1o8nYmovFevD27lfRRGDw1oWschT4F6koslisbx2eGAAYZOwl3DNh6iAhbbdmJ_cUbP9sCCPWnMYKRun-bnb5sGUxvlL1Z6VfvyN39qS0jcffm8Pup4hpeCpKrDhfxw'], 'User-Agent': ['Mozilla/5.0 (Macintosh; Intel Mac OS X 11_2_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36'], 'X-Amzn-Trace-Id': ['Root=1-60700f96-0ed0394d52ef733502b77e98'], 'X-Forwarded-For': ['100.33.115.103'], 'X-Forwarded-Port': ['443'], 'X-Forwarded-Proto': ['https']}, 'queryStringParameters': None, 'multiValueQueryStringParameters': None, 'pathParameters': None, 'stageVariables': None, 'requestContext': {'resourceId': 'tbw6vx', 'resourcePath': '/user/history', 'httpMethod': 'GET', 'extendedRequestId': 'dgdfhED8oAMFetg=', 'requestTime': '09/Apr/2021:08:25:58 +0000', 'path': '/v1/user/history', 'accountId': '387645926509', 'protocol': 'HTTP/1.1', 'stage': 'v1', 'domainPrefix': 'gr73qrcwnl', 'requestTimeEpoch': 1617956758494, 'requestId': 'a1620ec2-b052-4317-bd6b-6142f9ce763f', 'identity': {'cognitoIdentityPoolId': None, 'accountId': None, 'cognitoIdentityId': None, 'caller': None, 'sourceIp': '100.33.115.103', 'principalOrgId': None, 'accessKey': None, 'cognitoAuthenticationType': None, 'cognitoAuthenticationProvider': None, 'userArn': None, 'userAgent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 11_2_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36', 'user': None}, 'domainName': 'gr73qrcwnl.execute-api.us-east-1.amazonaws.com', 'apiId': 'gr73qrcwnl'}, 'body': None, 'isBase64Encoded': False}


In [128]:
access_token = event['headers']['access_token']

In [129]:
resp = cognito.get_user(
            AccessToken=access_token,
        )

NameError: name 'cognito' is not defined

In [169]:
data = [{'user_id': 'sy2938@columbia.edu', 'detail': 'I have question regarding to What is productize sticky markets, can anyone help?', 'created_at': Decimal('1618398000'), 'question_id': '6dc97eb6-9908-11eb-8788-acde48001122', 'question_status': 'accepted', 'start_time': '2021-04-16T19:00:00', 'tutor_id': 'montgomerycatherine@lewis.com', 'end_time': '2021-04-16T22:00:00', 'tags': ['Data Science'], 'title': 'What is productize sticky markets'}, {'user_id': 'sy2938@columbia.edu', 'detail': 'I have question regarding to What is revolutionize 24/365 paradigms, can anyone help?', 'created_at': Decimal('1617814380'), 'attachment': 'http://www.harper-hammond.com/', 'question_id': '6dc90bfc-9908-11eb-8788-acde48001122', 'question_status': 'posted', 'start_time': 'Null', 'tutor_id': 'Null', 'end_time': 'Null', 'tags': ['Chinese'], 'title': 'What is revolutionize 24/365 paradigms'}, {'user_id': 'sy2938@columbia.edu', 'detail': 'I have question regarding to What is mesh efficient eyeballs, can anyone help?', 'created_at': Decimal('1617909960'), 'question_id': '6dc98fc8-9908-11eb-8788-acde48001122', 'question_status': 'posted', 'start_time': 'Null', 'tutor_id': 'Null', 'end_time': 'Null', 'tags': [], 'title': 'What is mesh efficient eyeballs'}, {'user_id': 'sy2938@columbia.edu', 'detail': 'I have question regarding to What is envisioneer wireless paradigms, can anyone help?', 'created_at': Decimal('1618852560'), 'question_id': '6dc99a4a-9908-11eb-8788-acde48001122', 'question_status': 'posted', 'start_time': 'Null', 'tutor_id': 'Null', 'end_time': 'Null', 'tags': ['Chinese'], 'title': 'What is envisioneer wireless paradigms'}, {'user_id': 'sy2938@columbia.edu', 'detail': 'I have question regarding to What is deploy distributed e-commerce, can anyone help?', 'created_at': Decimal('1617904800'), 'question_id': '6dc98d16-9908-11eb-8788-acde48001122', 'question_status': 'confirmed', 'start_time': '2021-04-11T09:00:00', 'tutor_id': 'aaronpena@hotmail.com', 'end_time': '2021-04-11T10:00:00', 'tags': [], 'title': 'What is deploy distributed e-commerce'}, {'user_id': 'sy2938@columbia.edu', 'detail': 'I have question regarding to What is engineer plug-and-play partnerships, can anyone help?', 'created_at': Decimal('1617633300'), 'question_id': '6dc939ba-9908-11eb-8788-acde48001122', 'question_status': 'posted', 'start_time': 'Null', 'tutor_id': 'Null', 'end_time': 'Null', 'tags': ['Data Science'], 'title': 'What is engineer plug-and-play partnerships'}, {'user_id': 'sy2938@columbia.edu', 'detail': 'I have question regarding to What is whiteboard visionary info-mediaries, can anyone help?', 'created_at': Decimal('1619133960'), 'attachment': 'https://edwards.com/', 'question_id': '6dc97600-9908-11eb-8788-acde48001122', 'question_status': 'posted', 'start_time': 'Null', 'tutor_id': 'Null', 'end_time': 'Null', 'tags': [], 'title': 'What is whiteboard visionary info-mediaries'}, {'user_id': 'sy2938@columbia.edu', 'detail': 'I have question regarding to What is harness scalable infrastructures, can anyone help?', 'created_at': Decimal('1618023600'), 'attachment': 'http://www.hernandez.biz/', 'question_id': '6dc9317c-9908-11eb-8788-acde48001122', 'question_status': 'accepted', 'start_time': '2021-04-11T18:00:00', 'tutor_id': 'rachel81@west.com', 'end_time': '2021-04-11T21:00:00', 'tags': ['Chinese'], 'title': 'What is harness scalable infrastructures'}, {'user_id': 'sy2938@columbia.edu', 'detail': 'I have question regarding to What is synthesize bleeding-edge solutions, can anyone help?', 'created_at': Decimal('1617543900'), 'question_id': '6dc9607a-9908-11eb-8788-acde48001122', 'question_status': 'posted', 'start_time': 'Null', 'tutor_id': 'Null', 'end_time': 'Null', 'tags': ['Chinese'], 'title': 'What is synthesize bleeding-edge solutions'}, {'user_id': 'sy2938@columbia.edu', 'detail': 'I have question regarding to What is morph B2B applications, can anyone help?', 'created_at': Decimal('1618683720'), 'attachment': 'http://www.rasmussen.com/', 'question_id': '6dc9cfe2-9908-11eb-8788-acde48001122', 'question_status': 'posted', 'start_time': 'Null', 'tutor_id': 'Null', 'end_time': 'Null', 'tags': ['Data Science'], 'title': 'What is morph B2B applications'}]


In [154]:
user_ids = [i['user_id'] for i in data] + [i['tutor_id'] for i in data if i['tutor_id'] !='Null']

In [156]:
result = {i:{} for i in set(user_ids)}

In [165]:
response = dynamodb.batch_get_item(
    RequestItems={
        user_table:{
            "Keys": [{'user_id': user_id} for user_id in set(user_ids)],
            'AttributesToGet': ['user_id','picture', 'first_name', 'last_name']
        }
    }
)['Responses'][user_table]

result = {i['user_id']: i for i in response}

In [168]:
result['']

{'sy2938@columbia.edu': {'user_id': 'sy2938@columbia.edu',
  'last_name': 'Yuan',
  'picture': 'https://content.fakeface.rest/male_20_cb646615bf253a1351b1a04c44248768d565cbff.jpg',
  'first_name': 'Jackie'},
 'montgomerycatherine@lewis.com': {'user_id': 'montgomerycatherine@lewis.com',
  'last_name': 'Gujarati',
  'picture': 'https://content.fakeface.rest/male_25_c859d9567211e0d01d9cfcd44316d02cb6729723.jpg',
  'first_name': 'Casey'},
 'rachel81@west.com': {'user_id': 'rachel81@west.com',
  'last_name': 'Samoan',
  'picture': 'https://content.fakeface.rest/male_27_6bf83ca26a2eb71858817e7373dc1bd61cee4083.jpg',
  'first_name': 'Robert'},
 'aaronpena@hotmail.com': {'user_id': 'aaronpena@hotmail.com',
  'last_name': 'Japanese',
  'picture': None,
  'first_name': 'Karen'}}

In [166]:
for d in data:
    d['created_at'] = int(d['created_at'])
    d['user_picture'] = result[d['user_id']]['picture']
    d['user'] = result[d['user_id']]['first_name'] + ' ' + result[d['user_id']]['last_name']
    d['status'] = d['question_status']
    if d['tutor_id'] != 'Null':
        d['tutor_picture'] = result[d['tutor_id']]['picture']
        d['tutor'] = result[d['tutor_id']]['first_name'] + ' ' + result[d['tutor_id']]['last_name']
    else:
        d['tutor_id'] = None
        d['tutor_picture'] = None
        d['tutor'] = None
    d.pop('question_status', None)
    if 'tags' not in d:
        d['tags'] = []

In [167]:
data

[{'user_id': 'sy2938@columbia.edu',
  'detail': 'I have question regarding to What is productize sticky markets, can anyone help?',
  'created_at': 1618398000,
  'question_id': '6dc97eb6-9908-11eb-8788-acde48001122',
  'start_time': '2021-04-16T19:00:00',
  'tutor_id': 'montgomerycatherine@lewis.com',
  'end_time': '2021-04-16T22:00:00',
  'tags': ['Data Science'],
  'title': 'What is productize sticky markets',
  'user_picture': 'https://content.fakeface.rest/male_20_cb646615bf253a1351b1a04c44248768d565cbff.jpg',
  'user': 'Jackie Yuan',
  'status': 'accepted',
  'tutor_picture': 'https://content.fakeface.rest/male_25_c859d9567211e0d01d9cfcd44316d02cb6729723.jpg',
  'tutor': 'Casey Gujarati'},
 {'user_id': 'sy2938@columbia.edu',
  'detail': 'I have question regarding to What is revolutionize 24/365 paradigms, can anyone help?',
  'created_at': 1617814380,
  'attachment': 'http://www.harper-hammond.com/',
  'question_id': '6dc90bfc-9908-11eb-8788-acde48001122',
  'start_time': 'Null',


In [174]:
import time

In [176]:
time.time()

1617959822.599673